In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

# 1.5: Fault relations


Importing gempy


In [ ]:
import os

import gempy_viewer as gpv

# Aux imports
import numpy as np

import gempy as gp

np.random.seed(1515)

# 修复 OMP: Error #15 冲突
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

We import a model from an existing folder.


In [ ]:
data_path = os.path.abspath("..")

geo_model: gp.data.GeoModel = gp.create_geomodel(
    project_name="Faults_relations",
    extent=[0, 1000, 0, 1000, -1000, -400],
    resolution=[20, 20, 20],
    refinement=6,  # * For this model is better not to use octrees because we want to see what is happening in the scalar fields
    importer_helper=gp.data.ImporterHelper(
        path_to_orientations=data_path + "/data/tut_ch1-5_orientations.csv",
        path_to_surface_points=data_path + "/data/tut_ch1-5_points.csv",
    ),
)

print(geo_model)

## One fault model


Setting the structural frame


In [ ]:
fault1: gp.data.StructuralElement = geo_model.structural_frame.get_element_by_name("fault1")
fault2: gp.data.StructuralElement = geo_model.structural_frame.get_element_by_name("fault2")

# Remove the faults from the default group
default_group: gp.data.StructuralGroup = geo_model.structural_frame.get_group_by_name("default_formation")
default_group.elements.remove(fault1)
default_group.elements.remove(fault2)

# Add a new group for the fault
gp.add_structural_group(
    model=geo_model,
    group_index=0,
    structural_group_name="fault_series_1",
    elements=[fault1],
    structural_relation=gp.data.StackRelationType.FAULT,
    fault_relations=gp.data.FaultsRelationSpecialCase.OFFSET_ALL,
)

print(geo_model.structural_frame)

In [ ]:
geo_model.input_transform.apply_anisotropy(gp.data.GlobalAnisotropy.NONE)
gp.compute_model(geo_model)

In [ ]:
print(geo_model.solutions.raw_arrays.block_matrix[0])  # This contains the block values for the fault1
print(geo_model.solutions.raw_arrays.block_matrix[1])  # This contains the block values for the formations

In [ ]:
gpv.plot_2d(geo_model, show_data=True)

In [ ]:
gpv.plot_3d(geo_model, show_data=True, kwargs_plot_structured_grid={"opacity": 0.2})

In [ ]:
gp.add_structural_group(
    model=geo_model,
    group_index=1,
    structural_group_name="fault_series_2",
    elements=[fault2],
    structural_relation=gp.data.StackRelationType.FAULT,
    fault_relations=gp.data.FaultsRelationSpecialCase.OFFSET_ALL,
)
print(geo_model.structural_frame)

gp.compute_model(geo_model)

In [ ]:
gpv.plot_2d(geo_model, show_data=True)

In [ ]:
gpv.plot_3d(geo_model, show_data=True, image=True, kwargs_plot_structured_grid={"opacity": 0.2})

In [ ]:
gpv.plot_2d(geo_model, show_scalar=True, show_lith=False, series_n=0)
gpv.plot_2d(geo_model, show_scalar=True, show_lith=False, series_n=1)
gpv.plot_2d(geo_model, show_scalar=True, show_lith=False, series_n=2)

## Finite Faults


## Faults relations


Let's split the formations in two groups


In [ ]:
gp.add_structural_group(
    model=geo_model,
    group_index=2,
    structural_group_name="series_1",
    elements=[
        geo_model.structural_frame.get_element_by_name("rock4"),
        geo_model.structural_frame.get_element_by_name("rock3"),
    ],
    structural_relation=gp.data.StackRelationType.ERODE,
)

default_group.elements.remove(geo_model.structural_frame.get_element_by_name("rock4"))
default_group.elements.remove(geo_model.structural_frame.get_element_by_name("rock3"))

gp.set_fault_relation(
    frame=geo_model.structural_frame, rel_matrix=np.array([[0, 1, 1, 1], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]])
)
print(geo_model.structural_frame)

基于你提供的代码片段，特别是最后一个代码块中定义的断层关系矩阵（`rel_matrix`），这个教程最终构建的“有限断层”（Finite Fault）模型会有以下视觉和地质特征：

### 1. 核心特征：被掩埋的断层 (Fault 2)

在代码中，`fault_series_2` (即 **Fault 2**) 的关系矩阵行是 `[0, 0, 0, 1]`。

- **含义**：Fault 2 只错断了第 3 组地层（即最底层的 `default_formation`，包含 rock1 和 rock2）。它**没有**错断第 2 组地层（`series_1`，包含 rock3 和 rock4）。
- **视觉效果**：Fault 2 会表现为一个**有限断层**。你会看到它在深部地层（rock1, rock2）中造成了明显的位移，但断层面在向上延伸碰到 rock3 的底界时会**突然终止**。上覆的 rock3 和 rock4 地层是连续的、未被该断层破坏的。这在地质上通常代表断层活动停止后，发生了新的沉积事件（不整合）。

### 2. 贯穿断层 (Fault 1)

`fault_series_1` (即 **Fault 1**) 的关系矩阵行是 `[0, 1, 1, 1]`。

- **含义**：Fault 1 错断了所有其他组，包括 Fault 2、`series_1` 和 `default_formation`。
- **视觉效果**：Fault 1 是一个贯穿整个模型的年轻断层。它不仅切穿了所有的岩石层（从底部的 rock1 到顶部的 rock4），而且还切断了 Fault 2。

### 3. 整体几何形态

最终的模型看起来会像这样：

- **下部构造**：深部地层被 Fault 2 切割，形成某种地堑或地垒结构。
- **中部不整合面**：在深部构造之上，覆盖着一套完整的、未受 Fault 2 影响的地层（rock3, rock4）。
- **晚期构造**：整个这套“下部断层+上部覆盖层”的组合，又被一条新的大断层（Fault 1）整体切断并发生位移。

**总结来说，这个有限断层（Fault 2）看起来像是一个“盲断层”或“古断层”，它被上层年轻的沉积物所覆盖，只有在深部剖面中才能看到它的存在。**


In [ ]:
gp.compute_model(geo_model)

In [ ]:
gpv.plot_2d(geo_model, show_data=True)

In [ ]:
gpv.plot_3d(geo_model, show_data=True, image=False, kwargs_plot_structured_grid={"opacity": 0.2})